In [1]:
info = """
LangChain 1.0 - Memory Basics (内存管理基础)
==========================================

本模块重点讲解：
1. InMemorySaver - LangGraph 提供的内存管理
2. checkpointer 参数 - 为 Agent 添加内存
3. thread_id - 会话管理
4. 多轮对话状态保持
"""
print(info)



LangChain 1.0 - Memory Basics (内存管理基础)

本模块重点讲解：
1. InMemorySaver - LangGraph 提供的内存管理
2. checkpointer 参数 - 为 Agent 添加内存
3. thread_id - 会话管理
4. 多轮对话状态保持



In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
#  加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

# 初始化模型
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)


In [4]:
# 创建一个简单的工具
@tool
def get_user_info(user_id: str) -> str:
    """获取用户信息"""
    users = {
        "123": "张三，25岁，工程师",
        "456": "李四，30岁，设计师"
    }
    return users.get(user_id, "用户不存在")

In [6]:
"""
    示例1：没有内存的 Agent - 不记得之前的对话

    关键：每次调用都是独立的
    """
print("\n" + "="*70)
print("示例 1：没有内存的 Agent")
print("="*70)


示例 1：没有内存的 Agent


In [7]:

# 创建没有 checkpointer 的 Agent
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。"
)

In [8]:
print("\n第一轮对话：")
response1 = agent.invoke({
    "messages": [{"role": "user", "content": "我叫张三"}]
})
print(f"Agent: {response1['messages'][-1].content}")

print("\n第二轮对话：")
response2 = agent.invoke({
    "messages": [{"role": "user", "content": "我叫什么？"}]
})
print(f"Agent: {response2['messages'][-1].content}")



第一轮对话：
Agent: 很高兴认识你，张三。你需要帮助什么事情吗？我可以提供信息、回答问题，甚至帮助你完成任务。今天有什么可以帮你的吗？

第二轮对话：
Agent: 你没有告诉我你的名字！你想分享你的名字吗？我在这里帮助你，并且会记住你的名字以备将来参考。


In [9]:
print("\n关键点：")
print("  - Agent 不记得第一轮对话")
print("  - 每次 invoke 都是全新的开始")
print("  - 需要手动传入历史消息才能记住")


关键点：
  - Agent 不记得第一轮对话
  - 每次 invoke 都是全新的开始
  - 需要手动传入历史消息才能记住


In [10]:
"""
示例2：使用 InMemorySaver 添加短期内存

关键：
1. checkpointer=InMemorySaver()
2. config={"configurable": {"thread_id": "xxx"}}
"""
print("\n" + "="*70)
print("示例 2：使用 InMemorySaver 添加内存")
print("="*70)


示例 2：使用 InMemorySaver 添加内存


In [24]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    checkpointer=InMemorySaver()
)

In [25]:
config = {"configurable":{"thread_id":"conversation_1"}}

In [26]:
config

{'configurable': {'thread_id': 'conversation_1'}}

In [27]:
print("\n第一轮对话：")

response = agent.invoke({
    "messages": [{"role": "user", "content": "我叫张三"}],
    
},config)
response 


第一轮对话：


{'messages': [HumanMessage(content='我叫张三', additional_kwargs={}, response_metadata={}, id='93662d58-f44d-4cab-8427-b4da592b76f1'),
  AIMessage(content='很高兴认识你，张三。你今天过得怎么样？有什么需要帮助或想聊的话题吗？我在这里帮助你！', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 47, 'total_tokens': 79, 'completion_time': 0.167734415, 'completion_tokens_details': None, 'prompt_time': 0.002235602, 'prompt_tokens_details': None, 'queue_time': 0.054207707, 'total_time': 0.169970017}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3b42-f5b4-7f31-be77-a7bd1b385215-0', usage_metadata={'input_tokens': 47, 'output_tokens': 32, 'total_tokens': 79})]}

In [28]:
print("\n第二轮对话（同一个 thread_id）：")
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫什么？"}]},
    config=config  # 使用相同的 thread_id
)
print(f"Agent: {response2['messages'][-1].content}")


第二轮对话（同一个 thread_id）：
Agent: 你叫张三！


In [29]:
print("\n关键点：")
print("  - Agent 记住了第一轮对话！")
print("  - checkpointer 自动保存对话历史")
print("  - thread_id 用于区分不同的会话")


关键点：
  - Agent 记住了第一轮对话！
  - checkpointer 自动保存对话历史
  - thread_id 用于区分不同的会话


In [30]:
print("\n" + "="*70)
print("示例 3：多个独立会话")
print("="*70)


示例 3：多个独立会话


In [ ]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    checkpointer=InMemorySaver()
)

In [33]:
# 会话 1
config1 = {"configurable": {"thread_id": "user_alice"}}
print("\n[会话 1 - Alice]")
response = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫 Alice"}]},
    config=config1
)
print(f"Agent: {response['messages'][-1].content}")
print("Alice: 我叫 Alice")


[会话 1 - Alice]
Agent: 很高兴再次见到你，爱丽丝！你今天过得怎么样？有什么新鲜事吗？
Alice: 我叫 Alice


In [34]:
# 会话 2
config2 = {"configurable": {"thread_id": "user_bob"}}
print("\n[会话 2 - Bob]")
agent.invoke(
    {"messages": [{"role": "user", "content": "我叫 Bob"}]},
    config=config2
)
print("Bob: 我叫 Bob")

# 回到会话 1
print("\n[回到会话 1 - Alice]")
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫什么？"}]},
    config=config1
)
print(f"Agent: {response1['messages'][-1].content}")

# 回到会话 2
print("\n[回到会话 2 - Bob]")
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "我叫什么？"}]},
    config=config2
)


[会话 2 - Bob]
Bob: 我叫 Bob

[回到会话 1 - Alice]
Agent: 你叫爱丽丝！

[回到会话 2 - Bob]


In [ ]:
print(f"Agent: {response2['messages'][-1].content}")


Agent: 你叫鲍勃！我记得你刚刚告诉我的。


In [36]:
print("\n关键点：")
print("  - 不同 thread_id 的会话完全独立")
print("  - Agent 能正确记住每个会话的内容")
print("  - 适合多用户聊天场景")


关键点：
  - 不同 thread_id 的会话完全独立
  - Agent 能正确记住每个会话的内容
  - 适合多用户聊天场景


In [37]:
print("\n" + "="*70)
print("示例 4：内存 + 工具调用")
print("="*70)



示例 4：内存 + 工具调用


In [38]:
agent = create_agent(
    model=model,
    tools=[get_user_info],
    system_prompt="你是一个有帮助的助手。",
        checkpointer=InMemorySaver()
)

In [39]:
condfig = {"configurable": {"thread_id": "user_tool"}}

In [42]:
print("\n第一轮：查询用户信息")
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "查询用户 123 的信息"}]},
    config=config
)
print(f"Agent: {response1['messages'][-1].content}")


第一轮：查询用户信息
Agent: 用户 123 的信息是：张三，25岁，工程师


In [44]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "我刚才查询的用户年级多大？"}]
},config=config)
response

{'messages': [HumanMessage(content='查询用户 123 的信息', additional_kwargs={}, response_metadata={}, id='b5d47178-58b7-4358-ac2e-a1bba40466e7'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wbr6pcsg5', 'function': {'arguments': '{"user_id":"123"}', 'name': 'get_user_info'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 224, 'total_tokens': 240, 'completion_time': 0.034983644, 'completion_tokens_details': None, 'prompt_time': 0.020790593, 'prompt_tokens_details': None, 'queue_time': 0.056601346, 'total_time': 0.055774237}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3b45-e733-70f1-b5cc-a3c9fe05cf9e-0', tool_calls=[{'name': 'get_user_info', 'args': {'user_id': '123'}, 'id': 'wbr6pcsg5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 224, 'output_tokens': 16,

In [45]:

print("\n关键点：")
print("  - Agent 记住了工具调用的结果")
print("  - 不需要重新调用工具")
print("  - 对话上下文包含工具使用历史")


关键点：
  - Agent 记住了工具调用的结果
  - 不需要重新调用工具
  - 对话上下文包含工具使用历史


In [46]:
print("""
    示例5：查看和理解内存中保存的内容

    理解 checkpointer 保存了什么
    """)


    示例5：查看和理解内存中保存的内容

    理解 checkpointer 保存了什么
    


In [47]:
print("\n" + "="*70)
print("示例 5：查看内存状态")
print("="*70)


示例 5：查看内存状态


In [48]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
        checkpointer=InMemorySaver()
)

In [49]:
config = {"configurable": {"thread_id": "inspect_thread"}}


In [50]:
# 进行几轮对话
print("\n进行对话...")
agent.invoke(
    {"messages": [{"role": "user", "content": "你好"}]},
    config=config
)

agent.invoke(
    {"messages": [{"role": "user", "content": "我喜欢编程"}]},
    config=config
)

# 再次调用，查看返回的完整状态
response = agent.invoke(
    {"messages": [{"role": "user", "content": "我喜欢什么？"}]},
    config=config
)

print("\n对话历史中的消息数量:", len(response['messages']))


进行对话...

对话历史中的消息数量: 6


In [51]:
print("\n最近的消息：")
for msg in response['messages'][-3:]:
    msg_type = msg.__class__.__name__
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"  {msg_type}: {content}")


最近的消息：
  AIMessage: 那太棒了！编程是一项令人着迷的爱好和职业。它可以让你创造性地表达自己，开发创新解决方案，并解决现实世...
  HumanMessage: 我喜欢什么？
  AIMessage: 听起来你对编程的多个方面都很感兴趣。

如果你正在寻找一些具体的事情来探索，这里有一些建议：

1....


In [52]:
print("\n关键点：")
print("  - checkpointer 保存完整的消息历史")
print("  - response['messages'] 包含所有历史消息")
print("  - 每次调用都会追加新消息")



关键点：
  - checkpointer 保存完整的消息历史
  - response['messages'] 包含所有历史消息
  - 每次调用都会追加新消息


In [53]:
print("""
    示例6：实际应用场景 - 客服机器人

    模拟一个记住用户信息的客服场景
    """)


    示例6：实际应用场景 - 客服机器人

    模拟一个记住用户信息的客服场景
    


In [54]:
print("\n" + "="*70)
print("示例 6：实际应用 - 客服机器人")
print("="*70)



示例 6：实际应用 - 客服机器人


In [55]:
agent = create_agent(
        model=model,
        tools=[get_user_info],
        system_prompt="""你是一个客服助手。
特点：
- 记住用户说过的话
- 友好、有耐心
- 使用 get_user_info 工具查询用户信息时需要用户 ID""",
        checkpointer=InMemorySaver()
    )

In [56]:
# 模拟用户会话
user_id = "user_12345"
config = {"configurable": {"thread_id": user_id}}

In [63]:
conversations = [
    "你好，我想咨询一下",
    "我的用户 ID 是 123",
    "帮我查一下我的信息",
    "我多大来着？"  # 测试记忆
]

for i, user_msg in enumerate(conversations, 1):
    print(f"\n轮次 {i}:")
    print(f"用户: {user_msg}")

    response = agent.invoke(
        {"messages": [{"role": "user", "content": user_msg}]},
        config=config
    )

    print(f"客服: {response['messages'][-1].content}")
    print(response["messages"])
    print("*"*70)


轮次 1:
用户: 你好，我想咨询一下
客服: 你好！很高兴为你服务。今天我能如何帮助你呢？如果需要查询你的用户信息，我可能需要你的用户 ID，但我记得你之前的用户 ID 是 123。请问你是同一位张三先生吗？如果是的话，我可以直接为你查询信息。
[HumanMessage(content='你好，我想咨询一下', additional_kwargs={}, response_metadata={}, id='7f61974f-e19a-466d-8850-188eaa8e7f46'), AIMessage(content='你好！很高兴为你服务。今天我能如何帮助你呢？如果需要查询你的用户信息，我可能需要你的用户 ID，但现在我们可以先聊聊你想咨询什么。请随意分享你的想法或问题，我会尽力帮助你。', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 258, 'total_tokens': 317, 'completion_time': 0.350657253, 'completion_tokens_details': None, 'prompt_time': 0.013280708, 'prompt_tokens_details': None, 'queue_time': 0.053455712, 'total_time': 0.363937961}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3b4b-3735-7f91-82c7-fcfa5e2b6690-0', usage_metadata={'input_tokens': 258, 'output_tokens': 59, 'total_tokens': 317}), HumanMessage(content='我的用

In [ ]:
print("\n关键点：")
print("  - Agent 记住了用户的 ID")
print("  - Agent 记住了查询的结果")
print("  - 实现了流畅的多轮对话")